# Regression with Categorical Predictors - Part 1

This set of notes will explore using linear regression for a single predictor attribute that is categorical instead of continuous. To explore this first, let's explore some data. 

In [ ]:
library(tidyverse)
library(Lahman)
library(ggformula)

theme_set(theme_bw(base_size = 18))

career <- Batting %>%
  filter(AB > 100) %>%
  anti_join(Pitching, by = "playerID") %>%
  filter(yearID > 1990) %>%
  group_by(playerID, lgID) %>%
  summarise(H = sum(H), AB = sum(AB)) %>%
  mutate(average = H / AB)

career <- Master %>%
  tbl_df() %>%
  dplyr::select(playerID, nameFirst, nameLast) %>%
  unite(name, nameFirst, nameLast, sep = " ") %>%
  inner_join(career, by = "playerID") %>%
  dplyr::select(-playerID)

head(career)

## Question

Suppose we are interested in the batting average of baseball players since 1990, that is, the average is:

$$
average = \frac{number\ of\ hits}{number\ of\ atbats}
$$

Let's first visualize this.

In [ ]:
gf_density(~ average, data = career) %>%
  gf_labs(x = "Batting Average")

What if we hypothesized that the batting average will differ based on the league that players played in. 

In [ ]:
gf_violin(lgID ~ average, data = career, fill = 'gray80', draw_quantiles = c('0.1', '0.5', '0.9')) %>%
  gf_labs(x = "Batting Average",
          y = "League")

The distributions seem similar, but what if we wanted to go a step further and estimate a model to explore if there are really differences or not. For example, suppose we were interested in:

$$ 
H_{0}: \mu_{NL} = \mu_{AL}
$$

What type of model could we use? What about linear regression?

## Linear Regression with Categorical Attributes

Since these notes are happening, you can assume it is possible. But how can a categorical attribute with categories rather than numbers be included in the linear regression model?

The answer is that they can't. We need a new representation of the categorical attribute, enter dummy or indicator coding. 

### Dummy/Indicator Coding

Suppose we use the following logic:

If NL, then give a value of 1, else give a value of 0. 

Does this give the same information as before?

| League ID | Dummy League ID |
|-----------|-----------------|
| AL        | 0               | 
| NL        | 1               |

What would this look like for the actual data?

In [ ]:
career <- career %>%
  mutate(league_dummy = ifelse(lgID == 'NL', 1, 0))

head(career, n = 10)

Now that there is a numeric attribute, these can be added into the linear regression model.

In [ ]:
average_lm <- lm(average ~ league_dummy, data = career)

broom::tidy(average_lm)

How are these terms interpreted now?

In [ ]:
df_stats(average ~ league_dummy, data = career, mean, sd, length)

In [ ]:
average_lm2 <- lm(average ~ lgID, data = career)

broom::tidy(average_lm2)

In [ ]:
t.test(average ~ lgID, data = career, var.equal = TRUE)